In [406]:
##importing the required librarires

import pandas as pd
import numpy as np
import datetime as dt
import time
from alpha_vantage.timeseries import TimeSeries


In [407]:
class ScriptData(object):
    
    #initializing constructor 
    def __init__(self):
        self.main_var = pd.DataFrame()
        self.data = {}
    ## get method for cllecting json data 
    def __getJson__(self):
        return self.data
    ## set method for setting json data 
    def __setJson__(self,newkey):
            self.data =newkey
    ## set method for overloading dataframes for diffrent scripts
    def __setDataFrame__(self,newkey):
        self.main_var = newkey
    
    ## get method for getting dataframes for diffrent scripts
    def __getDataFrame__(self):
        return self.main_var
        
    ##this function will fetch data in json format
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key='PBUU95WNPEBUZYZI')
# Get json object with the intraday data and another with  the call's metadata
        data,meta = ts.get_intraday(script,"60min")
        self.__setJson__(data)
        #print(type(self.__getJson__()))
    
    ##this function will convert json data into pandas data frame
    def convert_intraday_data(self,script):
        df = pd.DataFrame.from_dict(self.data)
    
        df= df.transpose()
        df.reset_index(level = 0, inplace=True)
        df.columns = ["timestamp","open","high","low","close","volume"]
        df.sort_values(by='timestamp', inplace= True)
        df= df.reset_index()

        self.__setDataFrame__(df)## we are setting dataframeto main_varvariableofthe class which willconsist of the main data frame

In [408]:
script_data= ScriptData()## object of class ScriptData
script_data.fetch_intraday_data("NVDA")
script_data.convert_intraday_data("NVDA")
script_data.main_var
#script_data.plot(script_data.main_var)

,index,timestamp,open,high,low,close,volume
0,99,2022-12-27 17:00:00,141.2100,141.3000,140.2600,140.2999,442894
1,98,2022-12-27 18:00:00,140.3000,141.2100,140.1300,140.3500,85986
2,97,2022-12-27 19:00:00,140.3500,140.4800,140.2600,140.2700,24609
3,96,2022-12-27 20:00:00,140.2700,140.4000,139.9500,139.9900,63432
4,95,2022-12-28 05:00:00,141.2100,141.2100,138.7500,139.0000,44358
...,...,...,...,...,...,...,...
95,4,2023-01-05 16:00:00,143.8400,143.9800,142.4000,142.6600,5783153
96,3,2023-01-05 17:00:00,142.6500,142.9000,142.4410,142.8600,588476
97,2,2023-01-05 18:00:00,142.8100,142.8100,142.5600,142.7500,11943
98,1,2023-01-05 19:00:00,142.7800,142.8800,142.7000,142.7300,7452


In [409]:
###calculating moving average
### function to calculate moving average on close data point
def indicator1(df,timeperiod):
    temp= pd.DataFrame()
    temp["close"] = df["close"]
    temp["MA"] = temp["close"].rolling(timeperiod).mean()
    indicator_df = pd.DataFrame()
    indicator_df["timestamp"] = df["timestamp"]
    indicator_df["indicator"] = temp["MA"]
    return indicator_df

indicator1(script_data.main_var,5)

,timestamp,indicator
0,2022-12-27 17:00:00,NaN
1,2022-12-27 18:00:00,NaN
2,2022-12-27 19:00:00,NaN
3,2022-12-27 20:00:00,NaN
4,2022-12-28 05:00:00,139.98198
...,...,...
95,2023-01-05 16:00:00,142.92756
96,2023-01-05 17:00:00,143.10156
97,2023-01-05 18:00:00,143.13800
98,2023-01-05 19:00:00,142.96600


In [410]:
### defining our strategy 

In [411]:
class Strategy:
    def __init__(self,script):
        self.script = script
    ##fetching the required data
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
    
    ##function to geerate signals
    def get_signal(self):
        signals = pd.DataFrame()
        signals["timestamp"] = script_data.main_var["timestamp"]
        signals["signal"] = None
        close_data = script_data.main_var["close"]
        indicator_data = indicator1(script_data.main_var,5)["indicator"]
        temp = pd.DataFrame()###temproray dtaframe
        temp["prev_indicator"] = indicator_data.shift(1)
        temp["indicator"] = indicator_data
        temp["close"] = close_data
        temp = temp.astype(float)

            
        prev_indicator = indicator_data.shift(1)##prev indicator value in column
        prev_close = temp["close"].shift(1)##prev closevalue in column
        
        
        
        ##defining cut_upwards , cut_downward, 
        ## if the current indicator value is greater than close and previous indicator value is less then previous the it is a "BULLISH" crossover
        ##if the current indicator value is less than close and previous indicator value is greater then previous the it is a "BEARISH" crossover

        upward_crossing  = ((temp['indicator'] > temp['close']) & (prev_indicator < prev_close))## returns true if cndition is matced
        downward_crossing = ((temp['indicator'] < temp['close']) & (prev_indicator > prev_close))      
        

        temp["upward_crossing"] = upward_crossing
        temp["downward_crossing"]= downward_crossing
        #print(signals) 
        ## here we are assigning BUY and sell value if the value of upward_crossing& downward_Crossing is TRUE
        for i in range(len(temp["upward_crossing"])):
            if temp["upward_crossing"][i] == True:
                signals["signal"][i] = "BUY"
        for i in range(len(temp["downward_crossing"])):
            if temp["downward_crossing"][i] == True:
                signals["signal"][i] = "SELL"
            #if there is no buy and sell seignal we assign that column with "NO SISGNAL"
            elif signals["signal"][i] == None:
                signals["signal"][i] = "NO SIGNAL"
                
        signal_lst = ["BUY","SELL"]
        #print(temp)
        ##returning the dataframe with only buy and sell value 
        return signals[signals.signal.isin(signal_lst)]
        
        
strat = Strategy("NVDA")
strat.get_script_data()
strat.get_signal()

,timestamp,signal
5,2022-12-28 06:00:00,SELL
8,2022-12-28 09:00:00,BUY
9,2022-12-28 10:00:00,SELL
10,2022-12-28 11:00:00,BUY
12,2022-12-28 13:00:00,SELL
14,2022-12-28 15:00:00,BUY
18,2022-12-28 19:00:00,SELL
30,2022-12-29 15:00:00,BUY
31,2022-12-29 16:00:00,SELL
35,2022-12-29 20:00:00,BUY


In [412]:
#!pip install pyalgotrading

In [413]:
# from pyalgotrading.utils.candlesticks.heikinashi import HeikinAshi
# df = script_data.main_var
# df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
# df = df.astype({col: float for col in df.columns[1:]})



In [414]:
import plotly.graph_objects as go


In [415]:
df = script_data.main_var
indicator = indicator1(df,5)

chart_data = go.Candlestick(x=timestamps, open=df['open'], high=df['high'], low=df['low'], close=df['close'],name="Historical Data")
fig = go.Figure(data=[chart_data])
ema= go.Scatter(x=timestamps, y=indicator['indicator'], mode='lines', name='SMA')
fig.layout.xaxis.type = 'category'
fig.add_trace(ema)
fig.show()




In [416]:
# from pyalgotrading.constants import PlotType


# from pyalgotrading.utils import func

# new_indicator = indicator

# new_indicator['data'] = new_indicator['indicator']
# func.plot_candlestick_chart(df,PlotType.JAPANESE,hide_missing_dates= True)

In [417]:
# print(new_indicator)